## Inputs
---

In [1]:
# reading data and working with arrays
import os
import zarr, nrrd
import numpy as np
import os

# data paths
p = '/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/bigstream-testdata3'
fix_path = p + '/fix.n5'
ref_ch = 'c0'
fix_s0_subpath = f'{ref_ch}/s0'
mov_path = p + '/mov.n5'
mov_s0_subpath = f'{ref_ch}/s0'
exp_factor = 1

out_p = '/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-python'
os.makedirs(out_p, exist_ok=True)

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
fix_meta = fix_zarr[fix_s0_subpath].attrs.asdict()
print(f'Fix attrs: {fix_meta}')

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
mov_meta = mov_zarr[mov_s0_subpath].attrs.asdict()
print(f'Mov attrs: {mov_meta}')


Fix attrs: {'downsamplingFactors': [1.0, 1.0, 1.0], 'pixelResolution': {'dimensions': [1.0, 1.0, 1.0], 'unit': 'pixel'}}
Mov attrs: {'downsamplingFactors': [1.0, 1.0, 1.0], 'pixelResolution': {'dimensions': [1.0, 1.0, 1.0], 'unit': 'pixel'}}


In [2]:

fix_spacing_s0 = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing_s0 * [2, 2, 2]
fix_spacing_s2 = fix_spacing_s0 * [4, 4, 4]
fix_spacing_s3 = fix_spacing_s0 * [8, 8, 8]
fix_spacing_s4 = fix_spacing_s0 * [16, 16, 16]
fix_spacing_s5 = fix_spacing_s0 * [32, 32, 32]
print(f'fix_spacing_s0: {fix_spacing_s0}')
print(f'fix_spacing_s1: {fix_spacing_s1}')
print(f'fix_spacing_s2: {fix_spacing_s2}')
print(f'fix_spacing_s3: {fix_spacing_s3}')
print(f'fix_spacing_s4: {fix_spacing_s4}')
print(f'fix_spacing_s5: {fix_spacing_s5}')

mov_spacing_s0 = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing_s0 * [2, 2, 2]
mov_spacing_s2 = mov_spacing_s0 * [4, 4, 4]
mov_spacing_s3 = mov_spacing_s0 * [8, 8, 8]
mov_spacing_s4 = mov_spacing_s0 * [16, 16, 16]
mov_spacing_s5 = mov_spacing_s0 * [32, 32, 32]

print(f'mov_spacing_s0: {mov_spacing_s0}')
print(f'mov_spacing_s1: {mov_spacing_s1}')
print(f'mov_spacing_s2: {mov_spacing_s2}')
print(f'mov_spacing_s3: {mov_spacing_s3}')
print(f'mov_spacing_s4: {mov_spacing_s4}')
print(f'mov_spacing_s5: {mov_spacing_s5}')

global_scale = 's0'
fix_affine_spacing = fix_spacing_s0
mov_affine_spacing = mov_spacing_s0

local_scale = 's0'
fix_deform_spacing = fix_spacing_s0
mov_deform_spacing = mov_spacing_s0

fix_spacing_s0: [1. 1. 1.]
fix_spacing_s1: [2. 2. 2.]
fix_spacing_s2: [4. 4. 4.]
fix_spacing_s3: [8. 8. 8.]
fix_spacing_s4: [16. 16. 16.]
fix_spacing_s5: [32. 32. 32.]
mov_spacing_s0: [1. 1. 1.]
mov_spacing_s1: [2. 2. 2.]
mov_spacing_s2: [4. 4. 4.]
mov_spacing_s3: [8. 8. 8.]
mov_spacing_s4: [16. 16. 16.]
mov_spacing_s5: [32. 32. 32.]


## Alignment
---

### Compute Affine

In [3]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform


### Apply affine

In [4]:
# load precomputed results
affine = np.loadtxt(out_p+'/affine.mat')
affine

array([[ 9.84386100e-01, -4.44111772e-04,  1.01842376e-04,
         3.04607048e-02],
       [-1.39465551e-02,  9.96547376e-01, -6.27954979e-03,
         2.54243432e+01],
       [-5.16714859e-03, -1.13662287e-02,  9.97928648e-01,
        -4.42017886e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

### Prepare cluster args

In [5]:
cluster_kwargs={
    'config':{
        'distributed.worker.memory.target':0.9,
        'distributed.worker.memory.spill':0.9,
        'distributed.worker.memory.pause':0.9,
        'distributed.comm.retry.count':10,
        'distributed.comm.timeouts.connect':'600s',
        'distributed.scheduler.worker-saturation': 1,
        'distributed.scheduler.unknown-task-duration':'60m',
    },
}

### Calculate deform transformation

In [6]:
%%time
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline

# FASTER

# get full resolution data
fix = fix_zarr[f'/{ref_ch}/{local_scale}']
mov = mov_zarr[f'/{ref_ch}/{local_scale}']

# define alignment steps
affine_kwargs = {
    'smooth_sigmas':[0.25],
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 1.0,
}

deform_kwargs = {
    'smooth_sigmas':[0.25],
    'control_point_spacing':50.0,
    'control_point_levels':[1],
    'optimizer_args':{
        'learningRate':2.5,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 2.0,
}

steps = [ ('affine', affine_kwargs,), ('deform', deform_kwargs,), ]

# deform
deform = distributed_piecewise_alignment_pipeline(
    fix, mov,
    fix_deform_spacing, mov_deform_spacing,
    steps=steps,
    blocksize=[128, 128, 128],
    static_transform_list=[affine,],
    write_path=out_p+'/deform.zarr',
    cluster_kwargs=cluster_kwargs,
)


Partition (29, 266, 226) into [1 3 2] using [128 128 128] and [64 64 64]
Block index:  (0, 1, 0) 
Slices:  Block index:  (0, 0, 1) (slice(0, 29, None), slice(64, 266, None), slice(0, 192, None))
Slices:  
(slice(0, 29, None), slice(0, 192, None), slice(64, 226, None))
Block index: (0, 0, 1) - corner physical coords: [[  0.   0.  64.]
 [  0.   0. 225.]
 [  0. 191.  64.]
 [  0. 191. 225.]
 [ 28.   0.  64.]
 [ 28.   0. 225.]
 [ 28. 191.  64.]
 [ 28. 191. 225.]]
Block index: (0, 1, 0) - corner physical coords: [[  0.  64.   0.]
 [  0.  64. 191.]
 [  0. 265.   0.]
 [  0. 265. 191.]
 [ 28.  64.   0.]
 [ 28.  64. 191.]
 [ 28. 265.   0.]
 [ 28. 265. 191.]]
Sun Jun 16 14:29:33 2024 Block (0, 0, 1) :fix voxel coords [[  0   0  64]
 [  0   0 225]
 [  0 191  64]
 [  0 191 225]
 [ 28   0  64]
 [ 28   0 225]
 [ 28 191  64]
 [ 28 191 225]], fix phys coords [[  0.   0.  64.]
 [  0.   0. 225.]
 [  0. 191.  64.]
 [  0. 191. 225.]
 [ 28.   0.  64.]
 [ 28.   0. 225.]
 [ 28. 191.  64.]
 [ 28. 191. 225.]] -

/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bigstream/piecewise_align.py:387: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)


Sun Jun 16 14:29:49 2024 Block (0, 2, 1) :Apply weights (29, 74, 162), to transform (29, 74, 162, 3)
Sun Jun 16 14:29:49 2024 Finished computing (29, 74, 162, 3) block  transform in 15.688181161880493s (0, 2, 1)
Sun Jun 16 14:29:49 2024 Writing block (0, 2, 1) at (slice(0, 29, None), slice(192, 266, None), slice(64, 226, None))
Sun Jun 16 14:29:49 2024 Finished writing block (0, 2, 1) at (slice(0, 29, None), slice(192, 266, None), slice(64, 226, None))
(0, 1, 0) LEVEL: 0 ITERATION: 1  METRIC: -0.6363971891641589
(0, 0, 1) LEVEL: 0 ITERATION: 3  METRIC: -0.6734008013767974
(0, 0, 0) LEVEL: 0 ITERATION: 2  METRIC: -0.6291414598303297
Sun Jun 16 14:29:49 2024 Rebalance transform weights (0, 2, 0)
(0, 1, 1) LEVEL: 0 ITERATION: 2  METRIC: -0.6898719718926203


/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bigstream/piecewise_align.py:387: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)


Sun Jun 16 14:29:50 2024 Block (0, 2, 0) :Apply weights (29, 74, 192), to transform (29, 74, 192, 3)
Sun Jun 16 14:29:50 2024 Finished computing (29, 74, 192, 3) block  transform in 16.824720859527588s (0, 2, 0)
Sun Jun 16 14:29:50 2024 Writing block (0, 2, 0) at (slice(0, 29, None), slice(192, 266, None), slice(0, 192, None))
(0, 0, 1) LEVEL: 0 ITERATION: 4  METRIC: -0.6769534823591923
(0, 0, 0) LEVEL: 0 ITERATION: 3  METRIC: -0.6395820629476192
(0, 1, 0) LEVEL: 0 ITERATION: 2  METRIC: -0.6522383476316227
Sun Jun 16 14:29:50 2024 Finished writing block (0, 2, 0) at (slice(0, 29, None), slice(192, 266, None), slice(0, 192, None))
(0, 1, 1) LEVEL: 0 ITERATION: 3  METRIC: -0.6955201865781612
(0, 0, 1) LEVEL: 0 ITERATION: 5  METRIC: -0.6859509046343771
(0, 0, 0) LEVEL: 0 ITERATION: 4  METRIC: -0.6499396585667326
(0, 1, 0) LEVEL: 0 ITERATION: 3  METRIC: -0.6600766532021044
(0, 1, 1) LEVEL: 0 ITERATION: 4  METRIC: -0.7050714757766495
(0, 0, 1) LEVEL: 0 ITERATION: 6  METRIC: -0.6921006859158

...

Sun Jun 16 14:30:14 2024 Finished computing (29, 192, 192, 3) block  transform in 41.267855167388916s (0, 0, 0)
Sun Jun 16 14:30:14 2024 Writing block (0, 0, 0) at (slice(0, 29, None), slice(0, 192, None), slice(0, 192, None))
(0, 1, 0) LEVEL: 0 ITERATION: 22  METRIC: -0.7734984003402912
Sun Jun 16 14:30:15 2024 Finished writing block (0, 0, 0) at (slice(0, 29, None), slice(0, 192, None), slice(0, 192, None))
Sun Jun 16 14:30:15 2024 Rebalance transform weights (0, 1, 1)


/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bigstream/piecewise_align.py:387: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)


(0, 1, 0) LEVEL: 0 ITERATION: 23  METRIC: -0.7782366869394659
Sun Jun 16 14:30:15 2024 Block (0, 1, 1) :Apply weights (29, 202, 162), to transform (29, 202, 162, 3)
Sun Jun 16 14:30:15 2024 Finished computing (29, 202, 162, 3) block  transform in 41.92369723320007s (0, 1, 1)
Sun Jun 16 14:30:15 2024 Writing block (0, 1, 1) at (slice(0, 29, None), slice(64, 266, None), slice(64, 226, None))
Sun Jun 16 14:30:16 2024 Finished writing block (0, 1, 1) at (slice(0, 29, None), slice(64, 266, None), slice(64, 226, None))
(0, 1, 0) LEVEL: 0 ITERATION: 24  METRIC: -0.7809451185548433
Sun Jun 16 14:30:16 2024 (0, 1, 0) Registration succeeded
Sun Jun 16 14:30:16 2024 Completed (0, 1, 0) deform {'smooth_sigmas': [0.25], 'control_point_spacing': 50.0, 'control_point_levels': [1], 'optimizer_args': {'learningRate': 2.5, 'minStep': 0.0, 'numberOfIterations': 25}, 'alignment_spacing': 2.0, 'static_transform_list': [array([[ 9.84386100e-01, -4.44111772e-04,  1.01842376e-04,
         2.03755142e-03],
   

/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bigstream/piecewise_align.py:387: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)


Sun Jun 16 14:30:17 2024 Finished writing block (0, 1, 0) at (slice(0, 29, None), slice(64, 266, None), slice(0, 192, None))
CPU times: user 1.94 s, sys: 905 ms, total: 2.85 s
Wall time: 45.3 s


In [7]:
# read already saved result
deform = zarr.open(out_p+'/deform.zarr', mode='r')

In [8]:
print('!!!!!!Deform[0:10,0,0]',deform[0:10,0,0])

!!!!!!Deform[0:10,0,0] [[0.06829195 1.022542   1.5290565 ]
 [0.07427759 1.036904   1.5449455 ]
 [0.08043365 1.0498701  1.5581748 ]
 [0.08654308 1.0613896  1.568802  ]
 [0.09261805 1.0714942  1.5769047 ]
 [0.09867077 1.0802162  1.5825598 ]
 [0.10471343 1.0875878  1.5858451 ]
 [0.1107582  1.093641   1.5868376 ]
 [0.11681731 1.0984082  1.5856148 ]
 [0.12290294 1.1019213  1.5822538 ]]


In [9]:
from bigstream.piecewise_transform import distributed_apply_transform

blocksize = (64,)*3
channels = [ref_ch]

fix = fix_zarr[f'/{ref_ch}/{local_scale}']
for channel in channels:
    mov = mov_zarr[f'/{channel}/{local_scale}']
    aligned = distributed_apply_transform(
        fix, mov,
        fix_deform_spacing, mov_deform_spacing,
        transform_list=[affine, deform,],
        transform_spacing=((1,1,1), fix_deform_spacing),
        blocksize=blocksize,
        cluster_kwargs=cluster_kwargs
    )
    print(f'Write aligned result to:{out_p}/r2{channel}{local_scale}_aligned_to_r1.nrrd')
    nrrd.write(out_p+f'/r2{channel}{local_scale}_aligned_to_r1.nrrd', aligned.transpose(2,1,0), compression_level=2)


Block coords: (slice(0, 29, None), slice(224, 266, None), slice(96, 224, None)) Block origin: Block coords: (slice(0, 29, None), slice(160, 266, None), slice(0, 96, None)) Block origin: [  0. 224.  96.] Block size: [  0. 160.   0.] Block size: [64 64 64] Overlap: [64 64 64] Overlap: [32 32 32]
[32 32 32]
Transform slice for block at (slice(0, 29, None), slice(224, 266, None), slice(96, 224, None)): Transform slice for block at (slice(0, 29, None), slice(160, 266, None), slice(0, 96, None)): (slice(0, 29, None), slice(224, 266, None), slice(96, 224, None))
(slice(0, 29, None), slice(160, 266, None), slice(0, 96, None))
Block coords: (slice(0, 29, None), slice(224, 266, None), slice(32, 160, None)) Block origin: [  0. 224.  32.] Block size: [64 64 64] Overlap: Block coords: (slice(0, 29, None), slice(160, 266, None), slice(32, 160, None)) Block origin: [32 32 32]
Transform slice for block at (slice(0, 29, None), slice(224, 266, None), slice(32, 160, None)): (slice(0, 29, None), slice(224